In [1]:
from pyspark.sql import SparkSession
from datetime import date
from pyspark.sql.functions import lit, to_date

spark = SparkSession.builder.appName("data").getOrCreate()

individuals = spark.read.parquet("data/parquet/individual")
individuals = individuals.withColumn("observation_date", to_date(lit(date(2021, 5, 15))))
individuals.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/17 01:27:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+----------------+
|         customer_id|observation_date|
+--------------------+----------------+
|9b9a46fbf7ef42d99...|      2021-05-15|
|da1ded31d57e41898...|      2021-05-15|
|a8f58e8cfdce41b48...|      2021-05-15|
|d636a7e947614a09b...|      2021-05-15|
|3aad27a4cae64ca28...|      2021-05-15|
|21a1af01baa1438e8...|      2021-05-15|
|7b09cbd8954a4cfaa...|      2021-05-15|
|f83784bfd8064e2ea...|      2021-05-15|
|b0cded25f9f042ea9...|      2021-05-15|
|06d0e2ff15304e97b...|      2021-05-15|
|57eed97df817444d8...|      2021-05-15|
|d44297b089aa4603a...|      2021-05-15|
|8a3d852c76414c51b...|      2021-05-15|
|4b86373604e34ecca...|      2021-05-15|
|11e8a85589a54b279...|      2021-05-15|
|1d973b9861d54ec08...|      2021-05-15|
|63507ca4547e48f1b...|      2021-05-15|
|33405557128c4bb58...|      2021-05-15|
|5f03b9c48d574412a...|      2021-05-15|
|d89de3048ee24103b...|      2021-05-15|
+--------------------+----------------+
only showing top 20 rows



In [2]:
from supernova.metadata import FeatureStore

fs = FeatureStore.from_folder("docs/example")
fs

FeatureStore(entities=[Entity(name='individual', description='Entity representing a natural person or company', keys=['customer_id']), Entity(name='account', description='Entity representing an account within the company', keys=['branch_id', 'account_id']), Entity(name='conglomerate', description='A conglomerate of companies', keys=['conglomerate_id'])], feature_sets=[FeatureSet(name='Accounts Feature Set A', description='Feature set containing data about accounts for topic A', path='data/parquet/accounts_a', mnemonic='afsa', tags=[], entity=Entity(name='account', description='Entity representing an account within the company', keys=['branch_id', 'account_id']), date_field='calculation_date', features=[Feature(name='feature_a', description='Average monthly revolving debt in the last 6 months', type=<ParquetLogicalTypes.DOUBLE: 'DOUBLE'>, tags=[], domain=[RangeDomain(type='RANGE', start=0.0, include_start=False, end=inf, include_end=True, description='Values corresponding to the debt am

In [3]:
query = fs.query(["ifsa:*"])
query.execute(spark=spark, entity_df=individuals, pop_date_field="observation_date").show()

+--------------------+----------------+--------------------+--------------+--------------+--------------+--------------+---------------------+
|         customer_id|observation_date|    ifsa_customer_id|ifsa_feature_a|ifsa_feature_b|ifsa_feature_c|ifsa_feature_d|ifsa_calculation_date|
+--------------------+----------------+--------------------+--------------+--------------+--------------+--------------+---------------------+
|00a03d7989c9429fa...|      2021-05-15|00a03d7989c9429fa...|          5.39|          -1.0|             2|             1|           2021-05-14|
|00d384795fda425bb...|      2021-05-15|00d384795fda425bb...|          -1.0|          -1.0|            -1|             1|           2021-05-14|
|00d735df2aff4c218...|      2021-05-15|00d735df2aff4c218...|      89841.07|      90207.77|            -1|             0|           2021-05-14|
|01779326d7f44a3f9...|      2021-05-15|01779326d7f44a3f9...|      13952.48|          -1.0|             2|          NULL|           2021-05-14|